In [53]:
import nltk
import pandas as pd
import numpy as np
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import svm
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim_models as gmLDA
pyLDAvis.enable_notebook()

from matplotlib import pyplot as plt
%config InlineBackend.figure_formats = ['retina']  # or svg
%matplotlib inline
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('Resume.csv')
data.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [3]:
data = data[['Resume_str','Category']]
data.head()

,Resume_str,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,HR DIRECTOR Summary Over 2...,HR
3,HR SPECIALIST Summary Dedica...,HR
4,HR MANAGER Skill Highlights ...,HR


In [4]:
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

data['Resume_str'] = data.Resume_str.map(alphanumeric).map(punc_lower)
data.head()

,Resume_str,Category
0,hr administrator marketing associate\...,HR
1,hr specialist us hr operations ...,HR
2,hr director summary over ...,HR
3,hr specialist summary dedica...,HR
4,hr manager skill highlights ...,HR


In [62]:
data.Category.unique()

array(['HR', 'DESIGNER', 'INFORMATION-TECHNOLOGY', 'TEACHER', 'ADVOCATE',
       'BUSINESS-DEVELOPMENT', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE',
       'BPO', 'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE',
       'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT',
       'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION'],
      dtype=object)

In [5]:
X = data.Resume_str
y = data.Category
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
X[0]

'         hr administrator marketing associate\n\nhr administrator       summary     dedicated customer service manager with    years of experience in hospitality and customer service management    respected builder and leader of customer focused teams  strives to instill a shared  enthusiastic commitment to customer service          highlights         focused on customer satisfaction  team management  marketing savvy  conflict resolution techniques     training and development  skilled multi tasker  client relations specialist           accomplishments      missouri dot supervisor training certification  certified by ihg in customer loyalty and marketing by segment   hilton worldwide general manager training certification  accomplished trainer for cross server hospitality systems such as    hilton onq      micros    opera pms     fidelio    opera    reservation system  ors      holidex    completed courses and seminars in customer service  sales strategies  inventory control  loss pre

In [7]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(X)
print(dtm_tf.shape)

(2484, 7084)


In [16]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(X)
print(dtm_tfidf.shape)

(2484, 7084)


In [44]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=24, random_state=0)
lda_tf.fit(dtm_tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [45]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
12    -0.138861 -0.078934       1        1  9.526930
9     -0.003714  0.040656       2        1  7.751749
10     0.027865 -0.109680       3        1  6.823672
21     0.134873 -0.001778       4        1  6.404553
6     -0.152378  0.124701       5        1  5.859078
15     0.015046 -0.043452       6        1  5.445701
19    -0.080427 -0.158893       7        1  5.383272
20     0.158325 -0.018820       8        1  4.934226
1      0.096664 -0.026760       9        1  4.706944
0      0.014337  0.189948      10        1  4.659361
14    -0.036098  0.072723      11        1  3.953563
5     -0.160437 -0.185930      12        1  3.743658
23    -0.040917 -0.034800      13        1  3.603493
11     0.074943 -0.034870      14        1  3.597528
18    -0.040336  0.104710      15        1  3.575411
4     -0.040548 -0.057032      16        1  3.533398
2     -0.080074  0.167341      17        1  3.211187
22    -0.121927  0.132143      18        1  2.828554
13    -0.196714 -0.059270      19        1  2.737913
7      0.072509 -0.011578      20        1  2.618063
16     0.083507  0.047054      21        1  1.603245
17     0.167595 -0.031693      22        1  1.457562
8      0.040457 -0.021070      23        1  1.260451
3      0.206310 -0.004716      24        1  0.780487, topic_info=           Term         Freq        Total Category  logprob  loglift
3895  marketing  4479.000000  4479.000000  Default  30.0000  30.0000
2607  financial  4152.000000  4152.000000  Default  29.0000  29.0000
2677       food  2073.000000  2073.000000  Default  28.0000  28.0000
4970    project  5303.000000  5303.000000  Default  27.0000  27.0000
1838     design  3343.000000  3343.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
2303  equipment    40.029525  2433.121909  Topic24  -5.3951   0.7457
3340  insurance    35.923150  1061.759330  Topic24  -5.5033   1.4667
218      agency    31.845698   616.666746  Topic24  -5.6238   1.8896
4835   prepared    32.364923  1599.105613  Topic24  -5.6076   0.9529
3013     health    31.217123  2017.051996  Topic24  -5.6437   0.6846

[1697 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
0        13  0.094706            aaa
0        16  0.820787            aaa
0        17  0.063137            aaa
6        10  0.057569  abnormalities
6        22  0.345413  abnormalities
...     ...       ...            ...
7074     13  0.012860          youth
7074     18  0.021433          youth
7074     21  0.008573          youth
7076      1  0.963469            yoy
7083      3  0.974009           zoos

[10899 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 10, 11, 22, 7, 16, 20, 21, 2, 1, 15, 6, 24, 12, 19, 5, 3, 23, 14, 8, 17, 18, 9, 4])

In [58]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=24, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=24, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [67]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
18     -20.378456   56.508179       1        1  79.709196
14      79.288132   55.085800       2        1   7.691586
11      24.843275 -105.786751       3        1   4.128924
6       68.428604 -105.014183       4        1   1.528634
21      63.920837  -65.888359       5        1   0.721666
22      -8.527571  -64.856720       6        1   0.403195
13     -44.620640   19.916088       7        1   0.402279
12     -51.527081  -65.695885       8        1   0.352742
10      28.444641   -0.991994       9        1   0.344169
8       46.059116  -32.587276      10        1   0.328227
20     -19.095766 -102.703110      11        1   0.323781
9       66.543289    0.439861      12        1   0.323004
23      10.283104  -32.111134      13        1   0.318049
4      105.215370  -69.337166      14        1   0.317880
19     101.493652   17.312735      15        1   0.316582
1       29.093925   69.324188      16        1   0.315006
0       83.500191  -31.919847      17        1   0.313077
7       27.703207  -66.838219      18        1   0.310844
15     -10.297263    1.008217      19        1   0.308526
16     121.085579  -21.976244      20        1   0.308526
17     -27.011166  -31.074713      21        1   0.308526
5        9.422237   31.422739      22        1   0.308526
3       47.972301   31.643976      23        1   0.308526
2      -63.942307  -18.833242      24        1   0.308526, topic_info=            Term       Freq      Total Category  logprob  loglift
2199    employee  36.000000  36.000000  Default  30.0000  30.0000
2677        food  37.000000  37.000000  Default  29.0000  29.0000
6196    students  37.000000  37.000000  Default  28.0000  28.0000
3127       human  30.000000  30.000000  Default  27.0000  27.0000
5669      school  38.000000  38.000000  Default  26.0000  26.0000
...          ...        ...        ...      ...      ...      ...
896    cambridge   0.013483   0.674468  Topic24  -8.8656   1.8687
1805     density   0.013483   0.672725  Topic24  -8.8656   1.8712
4200       naval   0.013483   2.457408  Topic24  -8.8656   0.5757
4500         par   0.013483   0.978389  Topic24  -8.8656   1.4967
5315  relocating   0.013483   0.995642  Topic24  -8.8656   1.4792

[1193 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
29        2  0.935976   accessories
54        1  1.000403    accounting
55        1  0.997582      accounts
81        2  0.834770  acquisitions
94        1  0.878587    activities
...     ...       ...           ...
6942      5  0.260998       weapons
6943      3  0.561883          wear
6954      5  0.811608       wedding
7046      5  0.664817         wound
7080      5  0.863293          zone

[462 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 15, 12, 7, 22, 23, 14, 13, 11, 9, 21, 10, 24, 5, 20, 2, 1, 8, 16, 17, 18, 6, 4, 3])